Dans cet atelier nous allons entrainer un modèle machine learning avec Scikit-Learn et puis le déployer en tant que web service local et aussi en tant que web service ACI "Azure Container Instance".

Les étapes de cet Atelier sont les suivantes:

1. entrainer localement un modèle Scikit-learn
2. Suivre les expérimentations Scikit-learn avec MLFlow sur Azure Machine Learning
3. Enregistrer le modèle sur Azure Machine Learning
4. Déployer et tester le modèle en tant que web service local
5. Déployer et tester le modèle en tant que web service ACI "Azure Container Instance"

Tout au long de cet atelier nous allons utiliser Pima Indians Diabetes Database. Pour de plus amples informations sur les colonnes, visitez le lien ci-dessous:

https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.

## 1. L'entrainement du modèle Scikit-Learn

Assurez vous que vous avez installé anaconda avec une version de python 3.7.*

Assurez vous aussi que vous avez Docker installé : https://docs.docker.com/desktop/install/windows-install/

Dans le dossier de l'atelier, installez les dépendances requises: pip install -r requirements.txt

* azureml-core==1.39
* pandas==1.3.5
* scikit-learn==0.24.2
* cloudpickle==2.0.0
* psutil==5.9.0
* mlflow==1.24.0

In [106]:
pip install --upgrade mlflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install azure-core==1.21.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.1/178.1 kB 1.9 MB/s eta 0:00:00


In [4]:
pip install --upgrade azureml-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.0/933.0 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%clear history

In [107]:
import mlflow
from azureml.core import Workspace
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer

In [108]:
print(mlflow.__version__)


2.9.2


In [109]:
import warnings
warnings.filterwarnings('ignore')

Pour se connecter au workspace Azure machine learning, deux cas se présentent:

1. Télecharger le fichier de configuration et appeler le en utilisant ws = Workspace.from_config()

![image.png](attachment:image.png)

2. Ou bien en entrant manuellement les informations nécessaires, comme indiqué dans la cellule suivante.

In [110]:
#ws = Workspace.from_config()
subscription_id = '5da4cfa6-e95a-44e7-86f7-d99535671ea2'
resource_group = 'Learn_MLOps'
workspace_name = 'MLOpsS'
workspace_location="France Central"
ws = Workspace.create(name = workspace_name,location = workspace_location,resource_group = resource_group,
                             subscription_id = subscription_id,exist_ok=True)

#### Mettre en place un URI de Tracking des experimentation MLflow sur Azure

Nous indiquons à mlflow que l'URI de tracking est celui de mlflow dans azure ML. Toutefois, le lien de tracking de Mlflow dans azure à la forme suivante:

azureml://<region>.api.azureml.ms/mlflow/v1.0/subscriptions/<subscription-id>/resourceGroups/<resource-group>/providers/Microsoft.MachineLearningServices/workspaces/<aml-workspace>?

In [111]:
pip install azureml-mlflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip freeze

adal==1.2.7
alembic==1.13.1
anyio==4.2.0
argcomplete==3.2.1
asttokens==2.4.0
azure-common==1.1.28
azure-core==1.29.6
azure-graphrbac==0.61.1
azure-identity==1.15.0
azure-mgmt-authorization==4.0.0
azure-mgmt-containerregistry==10.3.0
azure-mgmt-core==1.4.0
azure-mgmt-keyvault==10.3.0
azure-mgmt-network==25.1.0
azure-mgmt-resource==23.0.1
azure-mgmt-storage==21.1.0
azure-storage-blob==12.19.0
azureml==0.2.7
azureml-core==1.54.0.post1
azureml-mlflow==1.54.0.post1
backcall==0.2.0
backports.tempfile==1.0
backports.weakref==1.0.post1
bcrypt==4.1.2
blinker==1.6.2
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.3
cloudpickle==3.0.0
colorama==0.4.6
comm==0.1.4
contextlib2==21.6.0
contourpy==1.2.0
cryptography==41.0.7
cycler==0.12.1
dash==2.9.3
dash-core-components==2.0.0
dash-html-components==2.0.0
dash-table==5.0.0
databricks-cli==0.18.0
debugpy==1.8.0
decorator==5.1.1
docker==6.1.3
entrypoints==0.4
exceptiongroup==1.1.3
executing==2.0.0
Flask==2.3.2
fonttools==4.47.0
git

In [112]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

Nous allons créer une expérimentation Mlflow dont le nom est "diabetes-sklearn"

In [113]:
pip install azureml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [114]:
experiment_name = 'diabetes-sklearn'
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', creation_time=1704542408333, experiment_id='c2def11d-b2f1-4564-946b-bfe00d6ec9cc', last_update_time=None, lifecycle_stage='active', name='diabetes-sklearn', tags={}>

#### chargement du dataset et traitements necéssaires

In [115]:
df=pd.read_csv('diabetes.csv')

In [116]:
df.shape

(768, 9)

In [117]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [119]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df_scaled=scaler.fit_transform(df)

In [120]:
df_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.46849198,
         1.4259954 ,  1.36589591],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.36506078,
        -0.19067191, -0.73212021],
       [ 1.23388019,  1.94372388, -0.26394125, ...,  0.60439732,
        -0.10558415,  1.36589591],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.68519336,
        -0.27575966, -0.73212021],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.37110101,
         1.17073215,  1.36589591],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.47378505,
        -0.87137393, -0.73212021]])

In [121]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1:]

In [122]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [123]:
print("X_train:",x_train.shape)
print("X_test:",x_test.shape)
print("Y_train:",y_train.shape)
print("Y_test:",y_test.shape)

X_train: (614, 8)
X_test: (154, 8)
Y_train: (614, 1)
Y_test: (154, 1)


In [124]:
rf_clf = RandomForestClassifier()

Au lieu d'appeler a chaque fois une fonction de logging (log_metric, log_artifact, ...) nous allons utiliser la fonction autolog() afin de logger toutes les informations relatives au modèle (metrics, parameters, ...)

les frameworks suivants supporte la fonction autolog():

    Scikit-learn

    TensorFlow and Keras

    Gluon

    XGBoost

    LightGBM

    Statsmodels

    Spark

    Fastai

    Pytorch



In [125]:
mlflow.sklearn.autolog(max_tuning_runs=None)

#### Hyperparameter tuning

Nous alons utiliser l'algorithme GridSearchCV pour trouver les meilleurs paramétres

In [126]:
param_grid = {'n_estimators': [10,20,30], 'max_depth':[2,7,10]}

In [127]:
gridcv=GridSearchCV(rf_clf,param_grid=param_grid,scoring = ['roc_auc', 'precision', 'recall', 'f1', 'accuracy'],refit = 'roc_auc',cv=10)

In [128]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [129]:
import matplotlib

In [130]:
gridcv.fit(x_train, y_train)

2024/01/06 22:50:12 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a267a99e-8965-44cc-93b4-221ae0087470', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/01/06 22:51:40 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID a267a99e-8965-44cc-93b4-221ae0087470. Failed operations: [RestException(\'BAD_REQUEST: Response: {\\\'Error\\\': {\\\'Code\\\': \\\'UserError\\\', \\\'Severity\\\': None, \\\'Message\\\': "Could not log all inputs as datasets:\\\\nCREATE can\\\'t modify existing entity, please use UPDATE.", \\\'MessageFormat\\\': None, \\\'MessageParameters\\\': None, \\\'ReferenceCode\\\': None, \\\'DetailsUri\\\': None, \\\'Target\\\': None, \\\'Details\\\': [], \\\'Inn

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 7, 10], 'n_estimators': [10, 20, 30]},
             refit='roc_auc',
             scoring=['roc_auc', 'precision', 'recall', 'f1', 'accuracy'])

#### visualisation des résultats sur Azure ML experiments

Les résultats du tracking et logging avec Mlflow peuvent être visualisés sur Azure machine learning studio en cliquant sur "Jobs" et puis sur le nom de l'experimentation (pour notre cas: diabetes-sklearn)

![image.png](attachment:image.png)

En cliquant sur le dernier job vous pouvez voir les résultats Mlflow de notre logging

![image-2.png](attachment:image-2.png)

les artifacts et run_id du meilleur modèle peuvent être trouvés en cliquent sur l'onglet "output+logs"

![image-3.png](attachment:image-3.png)



In [131]:
#modelLogistic = LogisticRegression()
#modelLogistic.fit(x_train,y_train)

In [132]:
#parameters={'penalty':['l1','l2'],"C":[1,2,3,4,5,10,100],'max_iter':[10,20,30]}

In [133]:
#mlflow.sklearn.autolog(max_tuning_runs=None)
#gridcv=GridSearchCV(modelLogistic,param_grid=parameters,scoring = ['roc_auc', 'precision', 'recall', 'f1', 'accuracy'],refit = 'roc_auc',cv=10)

In [134]:
#gridcv.fit(x_train,y_train)

## 2.  L'enregistrement du modèle
L'enregistrement du modèle dans le registre de modèles d'Azure Machine Learning a pour but de permettre aux utilisateurs de suivre les modifications apportées au modèle via la gestion des versions du modèle.

nous allons trouver l'expérimentation à partir du workspace en définant le workspace et le nom de l'expérimentation

In [135]:
from azureml.core import Experiment, Workspace
experiment_name = 'diabetes-sklearn'
experiment = Experiment(ws, experiment_name)

In [136]:
# afficher tous les runs
for r in experiment.get_runs():
    print(r)

Run(Experiment: diabetes-sklearn,
Id: a267a99e-8965-44cc-93b4-221ae0087470,
Type: None,
Status: Failed)
Run(Experiment: diabetes-sklearn,
Id: e8e8a305-fcef-407d-9dc1-a3d1ab1421a6,
Type: None,
Status: Failed)
Run(Experiment: diabetes-sklearn,
Id: 3831b5b8-04f5-4df9-9e28-9abac0b26bba,
Type: None,
Status: Completed)
Run(Experiment: diabetes-sklearn,
Id: fb1be63d-897b-4867-a163-697b55898b38,
Type: None,
Status: Running)
Run(Experiment: diabetes-sklearn,
Id: 7ab91901-682d-4024-9f4e-0bb1ec5fcccf,
Type: None,
Status: Running)
Run(Experiment: diabetes-sklearn,
Id: 47107f46-6c87-4d5a-b404-c684a1331346,
Type: None,
Status: Failed)
Run(Experiment: diabetes-sklearn,
Id: 32acc5ce-7a79-4f82-8c0a-be72fc3991f3,
Type: None,
Status: Failed)
Run(Experiment: diabetes-sklearn,
Id: f26008cd-ca79-48ac-bbb3-23bc17cfc05f,
Type: None,
Status: Failed)
Run(Experiment: diabetes-sklearn,
Id: 63714752-6423-4769-a887-a05c8f372c52,
Type: None,
Status: Failed)
Run(Experiment: diabetes-sklearn,
Id: fde2e13d-8f80-4abb-a2

Trouver le Run actuel

In [137]:
run_id = '3831b5b8-04f5-4df9-9e28-9abac0b26bba'
run = [r for r in experiment.get_runs() if r.id == run_id][0]

Enregistrer le modèle

    model_name: un nom arbitraire pour enregister le modèle
    model_path: chemin vers  model.pkl

In [138]:
model = run.register_model(model_name = 'diabetes_model', model_path = 'best_estimator/model.pkl')

on peut visualiser nos modèle enregistés en visitant "Models"

![image.png](attachment:image.png)

N.B: La dernière version enregistrée du modèle est 18

## 3. la création d'un script de Scoring

Le script de scoring généralement appelé score.py est utilisé lors de l'inférence comme point d'entrée du modèle.

score.py (voir le dossier de l'atelier) consiste en deux fonctions obligatoires:

1. init(): charge le modèle en tant que variable globale
2. run(): reçoit les nouvelles données à prédire à travers le paramètre data
      * effectue un pré-traitement des nouvelles données (optionnel)
      * effectue une prédiction sur les nouvelles données
      * effectue un post-traitement sur les prédictions (optionnel)
      * renvoie les résultats de la prédiction

## 4. Déploiement en local

Maintenant nous allons debuger le web service locallement avant de le déployer en production avec ACI " Azure container Instance"

Récupérez le modèle enregistré en définissant le workspace, le nom du modèle et la version du modèle.

In [139]:
from azureml.core.model import Model
model = Model(ws, 'diabetes_model', version=1)

### la création d'un environnement d'inférence personnalisé
Lors de l'entrainement du modèle, nous avons enregistré les dépendances de l'environnement dans MLFlow sous la forme d'un fichier conda.yaml. Nous utiliserons ce fichier pour créer un environnement d'inférence personnalisé.

![image.png](attachment:image.png)

Téléchargez le fichier conda.yml vers le dossier de l'atelier (déjà faite pour vous) et rajoutant d'autres dépendances comme: azureml-defaults==1.43, applicationinsights

le fichier conda.yml aura l'allure suivante (voir le dossier de l'atelier):

channels:
- conda-forge
dependencies:
- python=3.7.3
- pip
- pip:
  - mlflow
  - cffi==1.12.2
  - cloudpickle==2.0.0
  - pillow==5.4.1
  - psutil==5.9.0
  - scikit-learn==0.24.2
  - azureml-defaults==1.43
  - applicationinsights
name: mlflow-env

Nous allons nous baser sur ce fichier pour créer un environnement personnalisé

après enregistrement de cet environnement, nous pouvons le consulter dans la section "Environment" dans Azure Machine Learning et plus particulièrement dans l'onglet "Custom environments"

![image-2.png](attachment:image-2.png)

In [140]:
from azureml.core import Environment
env = Environment.from_conda_specification(name='diabetes-env6', file_path="./conda.yaml")
env.register(ws)

{
    "assetId": "azureml://locations/francecentral/workspaces/50554e45-0132-411e-8725-4bbd22a33172/environments/diabetes-env6/versions/6",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20231030.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "diabetes-env6",
 

### Définons la configuration d'inférence

In [141]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./score.py",
)

### Définons la configuration de déploiement

In [142]:
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6789)

### Déployons le service localement

Avant d'exécuter la cellule ci-dessous, assurez vous que Docker est démarré

In [145]:
service = Model.deploy(
    workspace = ws,
    name = 'diabetes-prediction-service1',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)

Generating Docker build context.
2024/01/06 22:15:47 Downloading source code...
2024/01/06 22:15:48 Finished downloading source code
2024/01/06 22:15:48 Creating Docker network: acb_default_network, driver: 'bridge'
2024/01/06 22:15:48 Successfully set up Docker network: acb_default_network
2024/01/06 22:15:48 Setting up Docker configuration...
2024/01/06 22:15:49 Successfully set up Docker configuration
2024/01/06 22:15:49 Logging in to registry: 50554e450132411e87254bbd22a33172.azurecr.io
2024/01/06 22:15:50 Successfully logged into 50554e450132411e87254bbd22a33172.azurecr.io
2024/01/06 22:15:50 Volume source scriptsFromEms successfully created
2024/01/06 22:15:50 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2024/01/06 22:15:50 Scanning for dependencies...
2024/01/06 22:15:51 Successfully scanned dependencies
2024/01/06 22:15:51 Launching container with name: acb_step_0
Sending build context to Docker daemon  77.31kB
Step 1/

WebserviceException: WebserviceException:
	Message: Package creation reached non-successful terminal state.
State: Failed
Error:
StatusCode: 400
Message: Environment image build failed.

	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Package creation reached non-successful terminal state.\nState: Failed\nError:\nStatusCode: 400\nMessage: Environment image build failed.\n"
    }
}

In [ ]:
service.wait_for_deployment(show_output=True)

Checking container health...
Local webservice is running at http://localhost:6789


Le fichier model.pkl sera téléchargé à partir d'Azure Machine Learning dans un dossier local temporaire et une image Docker avec les dépendances est créée et enregistrée dans Azure Container Registry (ACR). L'image sera téléchargée d'ACR sur la machine locale et un conteneur Docker exécutant le service Web est construit à partir de l'image localement.



Pour avoir l'URI de scoring lancez la commande suivante

In [ ]:
print (service.scoring_uri)

http://localhost:6789/score


## 5. Testons le service localement

Pour tester le service localement, ouvrez le notebook "Inference_test.ipynb" et assigner l'URI de scroring à la variable scoring_uri

Nous avons envoyé une demande d'inference au scoring_uri avec les données au format JSON. Voici à quoi ressemble input_data :

{"input": "[{\"Pregnancies\":6,\"Glucose\":148,\"BloodPressure\":72,\"SkinThickness\":35,\"Insulin\":0,\"BMI\":33.6,\"DiabetesPedigreeFunction\":0.627,\"Age\":50}]"}


Voici un exemple de la réponse pour l'inférence sur un seul enregistrement. La valeur de retour contient la probabilité qu'une personne reçoive un diagnostic de diabète.

prediction: "{\"proba\": [0.4829951216899814]}"


Le format de réponse peut être personnalisé dans la fonction run du fichier score.py.

## 6. Déplyons le service sur ACI

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
from azureml.core.model import Model
model = Model(ws, 'diabetes_model', version=1)
from azureml.core import Environment
env = Environment.get(workspace = ws, name = 'diabetes-env6', version = 1)

In [ ]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./score.py")

Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.


In [ ]:
from azureml.core.webservice import AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=0.1, memory_gb=0.5, auth_enabled=False)

In [ ]:
from azureml.core import Webservice

services = Webservice.list(ws)
for service in services:
    print(service.name)

In [ ]:
# Assuming 'service' is your deployed service object (ACI Webservice)
deployment_state = service.state
print(f"Service deployment state: {deployment_state}")

Service deployment state: None


In [ ]:
service.get_logs()

Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Sat, 06 Jan 2024 20:45:01 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': 'b601f0cb-2df2-4384-98dd-49ce409274f4', 'x-ms-client-session-id': '6541f682-13e0-4bd2-819f-237b4a6dd3f1', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-aml-cluster': 'vienna-francecentral-01', 'x-request-time': '0.047', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"NoSuchService","message":"There is no service with name: diabetes-prediction-service5 in Subscription: 5da4cfa6-e95a-44e7-86f7-d99535671ea2, ResourceGroup: Learn_MLOps, Workspace: MLOpsS, ACR: /subscriptions/5da4cfa6-e95a-44e7-

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Sat, 06 Jan 2024 20:45:01 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'x-ms-client-request-id': 'b601f0cb-2df2-4384-98dd-49ce409274f4', 'x-ms-client-session-id': '6541f682-13e0-4bd2-819f-237b4a6dd3f1', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-aml-cluster': 'vienna-francecentral-01', 'x-request-time': '0.047', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found.","details":[{"code":"NoSuchService","message":"There is no service with name: diabetes-prediction-service5 in Subscription: 5da4cfa6-e95a-44e7-86f7-d99535671ea2, ResourceGroup: Learn_MLOps, Worksp

In [51]:
pip install --upgrade azureml-sdk



  Obtaining dependency information for azureml-sdk from https://files.pythonhosted.org/packages/bc/e5/eada327875778790b9f543866f52c90e4bbb80e1c6e7deaaff6b6275aec6/azureml_sdk-1.54.0-py3-none-any.whl.metadata
  Obtaining dependency information for azureml-dataset-runtime[fuse]~=1.54.0 from https://files.pythonhosted.org/packages/2c/33/9b24f429dc492e56bf0044800a19878081da50b0c1c72d53d623f796069f/azureml_dataset_runtime-1.54.0.post1-py3-none-any.whl.metadata
  Obtaining dependency information for azureml-train-core~=1.54.0 from https://files.pythonhosted.org/packages/22/b0/6a3d282b242fb824f19b96e66be43b7d37e566d51777518243dfb8a7782b/azureml_train_core-1.54.0-py3-none-any.whl.metadata
  Obtaining dependency information for azureml-train-automl-client~=1.54.0 from https://files.pythonhosted.org/packages/8c/da/968e71982b72d1a3a26a432cba6c2a220c8f94d4cd79143c07db331b4b58/azureml_train_automl_client-1.54.0.post1-py3-none-any.whl.metadata
  Obtaining dependency information for azureml-pipeline

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\PC\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-65e29aac85b9409a6008e2dc84b1cc09.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [73]:
from azureml.core import Workspace

#Load the workspace from configuration
ws = Workspace.from_config()

#Access the workspace name

workspace_name = ws.name

print("Workspace Name:", workspace_name)

Workspace Name: MLOpsS


In [102]:
print(service.get_logs())

None


In [101]:
from azureml.core import Webservice

# Assuming 'service' is your deployed service object
service = Webservice(workspace=ws, name= 'diabetes-prediction-servicemh')

# Get the service state
service_state = service.state
print(f"Service state: {service_state}")

Service state: Failed


None


In [104]:
service = Model.deploy(
    workspace = ws,
    name = 'diabetes-prediction-servicemh1',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)
service.wait_for_deployment(show_output=True, timeout_sec=900)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-01-06 22:21:56+01:00 Creating Container Registry if not exists.
2024-01-06 22:21:56+01:00 Registering the environment.
2024-01-06 22:21:57+01:00 Use the existing image.
2024-01-06 22:21:57+01:00 Generating deployment configuration.
2024-01-06 22:21:58+01:00 Submitting deployment to compute.
2024-01-06 22:22:00+01:00 Checking the status of deployment diabetes-prediction-servicemh1..
2024-01-06 22:27:04+01:00 Checking the status of inference endpoint diabetes-prediction-servicemh1.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 3e50acad-f3e0-4a2a-95ef-2d6d8764be55
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: diabetes-prediction-servicemh1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 50554e450132411e87254bbd22a33172.azurecr.io/azureml/azureml_6e987b68a11bfde52cc4f950515e41a2 locally. Please refer to https://aka.ms/debugimage#se

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 3e50acad-f3e0-4a2a-95ef-2d6d8764be55
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: diabetes-prediction-servicemh1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 50554e450132411e87254bbd22a33172.azurecr.io/azureml/azureml_6e987b68a11bfde52cc4f950515e41a2 l

In [87]:
# Delete the existing service
existing_service = AciWebservice(workspace=ws, name='diabetes-prediction-servicemh')
existing_service.delete()

In [ ]:
#service.delete()

In [61]:
print (service.scoring_uri)

None


### Testons le service

Refaites la même chose en ouvrant le notebook "Inference_test.ipynb" et assigner le nouveau URI de scroring à la variable scoring_uri

## Déploiement sur AKS "Azure Kubernetes Service"

ACI est recommandé pour les tests sur une petite charge de travail de production. Pour une charge de travail importante azure fourni AKS "Azure Kubernetes Service"

Malheureusement le Déploiement sur AKS n'est pas pris en compte  avec notre sousription actuelle.

Néanmoins, je vous ai fourni le code de déploiement pour une éventuelle utilisation future

In [62]:
from azureml.core.webservice import AksWebservice
deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=False)

In [63]:
# ça suppose la création d'un kubernetes cluster
from azureml.core.compute import AksCompute
aks_target = AksCompute(ws,"myaks")

ComputeTargetException: ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name myaks not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name myaks not found in provided workspace"
    }
}

In [ ]:
service = Model.deploy(
    workspace = ws,
    name = 'diabetes-prediction-service-aks',
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    deployment_target=aks_target,
    overwrite=True)

service.wait_for_deployment(show_output=True)